In [16]:
from netCDF4 import Dataset,date2num,num2date
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [33]:
# Reading in the netCDF file
data=Dataset(r'C:/Users/Admin/Desktop/Test_py_ART/PHA2102082300.nc','r')
# Reading in station gauges information file
stations=pd.read_excel(pd.ExcelFile("C:/Users/Admin/Desktop/test_py_ART/stationLATLON.xlsx"))
# Making dates index
dates=num2date(data.variables['time'][:],units=data.variables['time'].units,calendar=data.variables['time'].calendar)
starting_date=str(dates[0])
ending_date=str(dates[-1])
date_range=pd.date_range(start=starting_date,end=ending_date,freq='H')
# Making a Dataframe
df=pd.DataFrame(0,columns=date_range,index=stations['Tên Trạm'])

In [12]:
#Extracting data from a netCDF file to dataframe
for index,row in stations.iterrows():
    locations=row['Tên Trạm']
    location_lat=row['Latitude']
    location_lon=row['Longitude']
    lat=data.variables['latitude'][:]
    lon=data.variables['longitude'][:]
    sq_diff_lat=(lat-location_lat)**2
    sq_diff_lon=(lon-location_lon)**2
    min_index_lat=sq_diff_lat.argmin()
    min_index_lon=sq_diff_lon.argmin()
    pre=data.variables['precipitation']
    for t_index in np.arange(0,len(date_range)):
        # calculation average in a cycle r=1km arround a location (gauge)
        cycle=[pre[t_index,min_index_lat,min_index_lon],pre[t_index,min_index_lat+1,min_index_lon],pre[t_index,min_index_lat-1,min_index_lon],pre[t_index,min_index_lat,min_index_lon+1],pre[t_index,min_index_lat,min_index_lon-1]]
        df.loc[[locations],date_range[t_index]]=np.nanmean(cycle)#pre[t_index,min_index_lat,min_index_lon]
df=df.fillna(0)

In [13]:
df

,2021-02-07 11:00:00,2021-02-07 12:00:00,2021-02-07 13:00:00,2021-02-07 14:00:00,2021-02-07 15:00:00,2021-02-07 16:00:00,2021-02-07 17:00:00,2021-02-07 18:00:00,2021-02-07 19:00:00,2021-02-07 20:00:00,...,2021-02-08 14:00:00,2021-02-08 15:00:00,2021-02-08 16:00:00,2021-02-08 17:00:00,2021-02-08 18:00:00,2021-02-08 19:00:00,2021-02-08 20:00:00,2021-02-08 21:00:00,2021-02-08 22:00:00,2021-02-08 23:00:00
Tên Trạm,,,,,,,,,,,,,,,,,,,,,
Chiềng Pha,0.000000,0.000000,0.007254,0.000000,0.000000,0.000000,0.000000,0.008681,0.000000,3.077666,...,0.901394,0.889057,0.519954,0.424943,0.748684,1.462659,2.047793,1.226079,0.871978,0.462023
Mường Sại,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024548,0.162263,1.405917,...,1.259044,1.407812,1.149774,0.956164,0.642442,1.945283,1.898694,0.560892,0.361163,0.518860
Ta Ma,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006854,3.521049,1.560290,0.926261,...,1.226234,0.368187,0.238648,0.281856,0.123599,0.395396,0.411469,0.784552,1.345489,0.677371
Chiềng Ơn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.193810,0.906418,1.582786,...,1.473106,1.082566,0.885960,1.310455,0.722078,1.560075,1.658632,0.993226,0.717784,0.890273
Chiềng Bôm,0.000000,0.010117,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.783460,...,1.893330,1.878558,1.210693,0.755038,1.718885,2.184759,2.192084,0.548183,0.490036,0.763388
Thuận Châu,0.000000,0.022784,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.042670,...,1.648666,1.769396,0.896660,0.900811,1.435849,2.169800,2.223095,0.922993,0.659787,0.808164
Mường Bám 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.829035,1.094464,0.730754,...,0.965282,0.943431,1.244648,0.647626,1.364979,1.823544,0.841479,0.199055,0.637341,0.677184
Mường Mùn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027213,3.712753,2.363129,0.464450,...,1.391388,1.076425,0.941452,0.779766,0.822364,0.800833,0.693848,0.947638,1.243451,0.672176
Phình Sáng,0.000000,0.000000,0.007234,0.014846,0.008246,0.006602,0.011414,1.154651,2.339798,0.584725,...,0.484382,0.780262,0.795076,0.407755,0.515497,0.806961,1.207930,0.980014,0.649849,0.594441


In [14]:
string='Pre_Radar_Phadin_Even1.xlsx'#+starting_date+"_"+ending_date+".xlsx"

In [15]:
# Making excel file
df.to_excel(string,float_format="%.2f")